<a href="https://colab.research.google.com/github/AlwaysEden/Machine-Learning-HW/blob/main/HW8_kNN_and_Random_Forest_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Mount your drive
from google.colab import drive
drive.mount('/content/drive')

# Code
1.   Implement and apply kNN on MNIST with k=1, 5, 10. Apply kNN on raw images, and 2, 7 dimensional eigenspaces, respectively. Show the accuracy scores for each run (you'd run the algorithm 9 times).

2.   Use and run the Random Forest algorithm for MNIST classification (http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html). Show the accuracy scores.

If it takes too much time, reduce the number of samples for training/testing (based on random selection).

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import urllib.request
import gzip
import pickle

In [125]:
def load_data(dataset):

    if not os.path.isfile(dataset):
        origin = (
            'http://www.iro.umontreal.ca/~lisa/deep/data/mnist/mnist.pkl.gz'
        )
        print('Downloading data from %s' % origin)
        urllib.request.urlretrieve(origin, dataset)

    print('Loading data...')

    with gzip.open(dataset, 'rb') as f:
        try:
            train_set, valid_set, test_set = pickle.load(f, encoding='latin1')
        except:
            train_set, valid_set, test_set = pickle.load(f)

    all_data_x = np.concatenate([train_set[0], valid_set[0], test_set[0]])
    all_data_y = np.concatenate([train_set[1], valid_set[1], test_set[1]])

    print('... data has been loaded!')
    return all_data_x, all_data_y

def PCA(X, n_components):
    # Step 1: Center the data
    X_centered = X - np.mean(X, axis=0)

    # Step 2: Compute the covariance matrix
    covariance_matrix = np.cov(X_centered, rowvar=False)

    # Step 3: Eigen decomposition
    eigenvalues, eigenvectors = np.linalg.eigh(covariance_matrix)

    # Step 4: Sort eigenvectors by eigenvalues in descending order
    sorted_indices = np.argsort(eigenvalues)[::-1]
    sorted_eigenvectors = eigenvectors[:, sorted_indices]

    # Step 5: Select the top 'n_components' eigenvectors
    W = sorted_eigenvectors[:, :n_components]

    # Step 6: Project the data onto the new feature space
    X_pca = X_centered.dot(W)

    return X_pca

def euclidean_distance(x1, x2):
  return np.sqrt(np.sum((x1 - x2) ** 2))

def splitData(data_x, data_y):
  train_size = int(0.7 * data_x.shape[0])
  val_size = int(0.15 * data_x.shape[0])
  test_size = data_x.shape[0] - train_size - val_size

  #Random
  indices = np.arange(data_x.shape[0])
  np.random.shuffle(indices)
  data_x = data_x[indices]
  data_y = data_y[indices]

  train_x = data_x[:train_size]
  train_y = data_y[:train_size]
  val_x = data_x[train_size:train_size+val_size]
  val_y = data_y[train_size:train_size+val_size]
  test_x = data_x[train_size+val_size:]
  test_y = data_y[train_size+val_size:]

  print("... data has been splited")
  return train_x, train_y, val_x, val_y, test_x, test_y

def accuracy(y_true, y_pred):
  correct = np.sum(y_true == y_pred)
  return correct / len(y_true)

def kNN(target_x, target_y, k):
  arr = np.zeros((target_x.shape[0],target_x.shape[0]))
  arr[np.diag_indices_from(arr)] = 10000 # (n,n) = 10000 for distinguishing

  neighbors = np.zeros((target_x.shape[0],k))

  #Look for k neighbors.
  for i in range(target_x.shape[0]):
    for j in range(i+1, target_x.shape[0]):
      arr[i][j] = euclidean_distance(target_x[i], target_x[j])
      arr[j][i] = arr[i][j] #For reduction of duplicate computation
    for x in range(k):
      neighbors[i][x] = np.argmin(arr[i])
      arr[i][int(neighbors[i][x])] = 10000

  #Classification Vote
  pred_y = []
  for i in range(target_x.shape[0]):
    new = []
    for j in range(k):
      new.append(target_y[int(neighbors[i][j])])
    tmp_arr, count = np.unique(new, return_counts=True)
    pred_y.append(tmp_arr[np.argmax(count)])

  return pred_y

In [130]:
data_x, data_y= load_data('mnist.pkl.gz')
train_x, train_y, val_x, val_y, test_x, test_y = splitData(data_x, data_y)

Loading data...
... data has been loaded!
... data has been splited


In [108]:
#kNN

''' If you want to reduce the number of data
indices = np.arange(data_x.shape[0])
np.random.shuffle(indices)

data_x = data_x[indices]
data_y = data_y[indices]
data_x = data_x[:1000]
data_y = data_y[:1000]
'''
# k = 1, raw_images
pred_y = kNN(data_x, data_y, 1)
print(f"k=1 & raw_image => acc = {accuracy(data_y, pred_y)}")

# k = 1, 2D
data_x_pca = PCA(data_x, 2)
pred_y = kNN(data_x_pca, data_y, 1)
print(f"k=1 & 2D => acc = {accuracy(data_y, pred_y)}")

# k = 1, 7D
data_x_pca = PCA(data_x, 7)
pred_y = kNN(data_x_pca, data_y, 1)
print(f"k=1 & 7D => acc = {accuracy(data_y, pred_y)}")

# k = 5, raw_images
pred_y = kNN(data_x, data_y, 5)
print(f"k=5 & raw_image => acc = {accuracy(data_y, pred_y)}")

# k = 5, 2D
data_x_pca = PCA(data_x, 2)

pred_y = kNN(data_x_pca, data_y, 1)
print(f"k=5 & 2D => acc = {accuracy(data_y, pred_y)}")

# k = 5, 7D
data_x_pca = PCA(data_x, 7)

pred_y = kNN(data_x_pca, data_y, 1)
print(f"k=5 & 7D => acc = {accuracy(data_y, pred_y)}")

# k = 10, raw_images
pred_y = kNN(data_x, data_y, 10)
print(f"k=10 & raw_image => acc = {accuracy(data_y, pred_y)}")

# k = 10, 2D
data_x_pca = PCA(data_x, 2)
pred_y = kNN(data_x_pca, data_y, 10)
print(f"k=10 & 2D => acc = {accuracy(data_y, pred_y)}")

# k = 10, 7D
data_x_pca = PCA(data_x, 7)
pred_y = kNN(data_x_pca, data_y, 10)
print(f"k=10 & 7D => acc = {accuracy(data_y, pred_y)}")

k=1 & raw_image => acc = 0.899
k=1 & 2D => acc = 0.399
k=1 & 7D => acc = 0.77
k=5 & raw_image => acc = 0.87
k=5 & 2D => acc = 0.399
k=5 & 7D => acc = 0.77
k=10 & raw_image => acc = 0.854
k=10 & 2D => acc = 0.437
k=10 & 7D => acc = 0.81


In [135]:
from sklearn.ensemble import RandomForestClassifier

''' If you want to reduce the number of train, val, and test data.
n = 1000 #the number of data

indices = np.arange(train_x.shape[0])
np.random.shuffle(indices)

train_x = train_x[indices]
train_y = train_y[indices]
train_x = train_x[:n]
train_y = train_y[:n]

indices = np.arange(val_x.shape[0])
np.random.shuffle(indices)

val_x = val_x[indices]
val_y = val_y[indices]
val_x = val_x[:n]
val_y = val_y[:n]

indices = np.arange(test_x.shape[0])
np.random.shuffle(indices)

test_x = test_x[indices]
test_y = test_y[indices]
test_x = test_x[:n]
test_y = test_y[:n]
'''

#raw_image
rf_classifier = RandomForestClassifier(n_estimators=100, oob_score=True, random_state=42) # n_estimators는 트리 개수, random_state는 랜덤 시드
rf_classifier.fit(train_x, train_y)
y_pred_val = rf_classifier.predict(val_x)
y_pred_test = rf_classifier.predict(test_x)
accuracy_val = accuracy(val_y, y_pred_val)
accuracy_test = accuracy(test_y, y_pred_test)
oob_score = rf_classifier.oob_score_
print(f"OOB Score(raw_image): {oob_score}")
print(f"val_acc(raw_image): {accuracy_val}")
print(f"test_acc(raw_image): {accuracy_test}")


#2D
rf_classifier = RandomForestClassifier(n_estimators=300, oob_score=True, random_state=42) # n_estimators는 트리 개수, random_state는 랜덤 시드
train_x_pca = PCA(train_x, 2)
val_x_pca = PCA(val_x, 2)
test_x_pca = PCA(test_x, 2)
rf_classifier.fit(train_x_pca, train_y)
y_pred_val = rf_classifier.predict(val_x_pca)
y_pred_test = rf_classifier.predict(test_x_pca)
accuracy_val = accuracy(val_y, y_pred_val)
accuracy_test = accuracy(test_y, y_pred_test)
oob_score = rf_classifier.oob_score_
print(f"OOB Score(2D): {oob_score}")
print(f"val_acc(2D): {accuracy_val}")
print(f"test_acc(2D): {accuracy_test}")

#7D
rf_classifier = RandomForestClassifier(n_estimators=300, oob_score=True,random_state=42) # n_estimators는 트리 개수, random_state는 랜덤 시드
train_x_pca = PCA(train_x, 7)
val_x_pca = PCA(val_x, 7)
test_x_pca = PCA(test_x, 7)
rf_classifier.fit(train_x_pca, train_y)
y_pred_val = rf_classifier.predict(val_x_pca)
y_pred_test = rf_classifier.predict(test_x_pca)
accuracy_val = accuracy(val_y, y_pred_val)
accuracy_test = accuracy(test_y, y_pred_test)
oob_score = rf_classifier.oob_score_
print(f"OOB Score(7D): {oob_score}")
print(f"val_acc(7D): {accuracy_val}")
print(f"test_acc(7D): {accuracy_test}")

OOB Score(raw_image): 0.89
val_acc(raw_image): 0.902
test_acc(raw_image): 0.906
OOB Score(2D): 0.419
val_acc(2D): 0.364
test_acc(2D): 0.116
OOB Score(7D): 0.801
val_acc(7D): 0.524
test_acc(7D): 0.192


# ChatGPT prompts

Now solve the same task with ChatGPT with the following prompts:


1.   Can you implement and apply kNN on MNIST (from scratch without using any specialized library) with k=1, 5, 10? Can you show the accuracies of the predictions? Ignore the computational cost of this code.
2.   Can you use the same provided knn algorithm and apply it to the MNIST data with reduced dimensions to 2 and 7, with k=1, 5, 10? Please provide the accuracies.
3.   Can you provide a code for the random forest algorithm applied to the MNIST dataset? Please use specialized libraries like sklearn. Please show the accuracies scores.


**Paste the answer provided and run the code below:**

In [142]:
#ChatGPT Prompts 1
import numpy as np
from collections import Counter

def load_data(dataset):

    if not os.path.isfile(dataset):
        origin = (
            'http://www.iro.umontreal.ca/~lisa/deep/data/mnist/mnist.pkl.gz'
        )
        print('Downloading data from %s' % origin)
        urllib.request.urlretrieve(origin, dataset)

    print('Loading data...')

    with gzip.open(dataset, 'rb') as f:
        try:
            train_set, valid_set, test_set = pickle.load(f, encoding='latin1')
        except:
            train_set, valid_set, test_set = pickle.load(f)

    print('... data has been loaded!')
    return train_set, valid_set, test_set

# k-NN function to make predictions
def knn_predict(X_train, y_train, X_test, k):
    predictions = []
    for test_point in X_test:
        # Compute distances to all training points
        distances = np.sqrt(np.sum((X_train - test_point) ** 2, axis=1))

        # Find the k nearest neighbors and their labels
        k_indices = np.argsort(distances)[:k]
        k_nearest_labels = y_train[k_indices]

        # Predict the most common label
        most_common = Counter(k_nearest_labels).most_common(1)[0][0]
        predictions.append(most_common)

    return np.array(predictions)

# Function to evaluate the k-NN algorithm for different k values
def evaluate_knn(X_train, y_train, X_test, y_test, k_values):
    accuracies = {}
    for k in k_values:
        y_pred = knn_predict(X_train, y_train, X_test, k)
        accuracy = np.mean(y_pred == y_test) * 100  # Calculate accuracy as a percentage
        accuracies[k] = accuracy
        print(f"Accuracy for k={k}: {accuracy:.2f}%")
    return accuracies

# Example usage with training and test data
# X_train, y_train, X_test, y_test should be defined beforehand
train_set, val_set, test_set = load_data('mnist.pkl.gz')
X_train, y_train = train_set
X_test, y_test = test_set

k_values = [1, 5, 10]
accuracies = evaluate_knn(X_train, y_train, X_test, y_test, k_values)


Loading data...
... data has been loaded!
Accuracy for k=1: 87.90%
Accuracy for k=5: 87.80%
Accuracy for k=10: 87.30%


In [141]:
#ChatGPT Prompts 2
import numpy as np
from collections import Counter

# Function to reduce dimensions with PCA
def reduce_dimensions(X, n_components):
    # Center the data
    X_centered = X - np.mean(X, axis=0)
    # Compute covariance matrix
    covariance_matrix = np.cov(X_centered, rowvar=False)
    # Compute eigenvalues and eigenvectors
    eigenvalues, eigenvectors = np.linalg.eigh(covariance_matrix)
    # Select the top 'n_components' eigenvectors
    sorted_indices = np.argsort(eigenvalues)[::-1]
    top_eigenvectors = eigenvectors[:, sorted_indices[:n_components]]
    # Project data onto top eigenvectors
    return np.dot(X_centered, top_eigenvectors)

# k-NN function to make predictions
def knn_predict(X_train, y_train, X_test, k):
    predictions = []
    for test_point in X_test:
        # Compute distances to all training points
        distances = np.sqrt(np.sum((X_train - test_point) ** 2, axis=1))
        # Find the k nearest neighbors and their labels
        k_indices = np.argsort(distances)[:k]
        k_nearest_labels = y_train[k_indices]
        # Predict the most common label
        most_common = Counter(k_nearest_labels).most_common(1)[0][0]
        predictions.append(most_common)
    return np.array(predictions)

# Function to evaluate the k-NN algorithm for different k values
def evaluate_knn(X_train, y_train, X_test, y_test, k_values):
    accuracies = {}
    for k in k_values:
        y_pred = knn_predict(X_train, y_train, X_test, k)
        accuracy = np.mean(y_pred == y_test) * 100  # Calculate accuracy as a percentage
        accuracies[k] = accuracy
        print(f"Accuracy for k={k}: {accuracy:.2f}%")
    return accuracies

# Example usage with dimensionally reduced data
# X_train, y_train, X_test, y_test should be defined beforehand
train_set, val_set, test_set = load_data('mnist.pkl.gz')
X_train, y_train = train_set
X_test, y_test = test_set
# Define k values to evaluate
k_values = [1, 5, 10]

# Reduce MNIST dataset dimensions to 2 and evaluate k-NN
X_train_reduced_2 = reduce_dimensions(X_train, 2)
X_test_reduced_2 = reduce_dimensions(X_test, 2)
print("Results for 2 dimensions:")
accuracies_2 = evaluate_knn(X_train_reduced_2, y_train, X_test_reduced_2, y_test, k_values)

# Reduce MNIST dataset dimensions to 7 and evaluate k-NN
X_train_reduced_7 = reduce_dimensions(X_train, 7)
X_test_reduced_7 = reduce_dimensions(X_test, 7)
print("\nResults for 7 dimensions:")
accuracies_7 = evaluate_knn(X_train_reduced_7, y_train, X_test_reduced_7, y_test, k_values)


Loading data...
... data has been loaded!
Results for 2 dimensions:
Accuracy for k=1: 3.60%
Accuracy for k=5: 3.60%
Accuracy for k=10: 4.40%

Results for 7 dimensions:
Accuracy for k=1: 2.30%
Accuracy for k=5: 1.50%
Accuracy for k=10: 1.20%


In [144]:
#ChatGPT prompts 3
from sklearn.datasets import fetch_openml
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Load the MNIST dataset
mnist = fetch_openml("mnist_784", version=1)
X = mnist.data
y = mnist.target.astype(int)

# Split dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = rf_classifier.predict(X_test)

# Calculate and print the accuracy score
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy of Random Forest on MNIST dataset: {accuracy * 100:.2f}%")

Accuracy of Random Forest on MNIST dataset: 89.90%


# Report


1.   What are some differences you notice between your code and ChatGPT's? (Name at least 3)
2.   Ask ChatGPT to compare kNN and Random Forest. Paste the answer here. Do you agree with ChatGPT's statement? Why yes or why not?
3.   In yours and ChatGPTs implementation, which algorithm had the highest accuracy?
4.   Conclude with some thoughts and things you learned from this homework.

# Answer
1.
- Majority voting is different.
- Scores are measured differently
- Different ways to find neighbors
2. I agree with most GPT's idea. However, I have some thought about a few idea. First, ChatGPT said that kNN has a very fast training time. It could be possible in terms of training time but kNN algorithm take a lot of time in terms of whole algorithm because of looking for k neighbors. Second, ChatGPT said that kNN struggles with high-dimensional data. Although ChatGPT said it, kNN show me high performance using raw images.
3. My Random Forest has the highest accuracy. Surely the difference between my RF and ChatGPT's RF is 1%.
4. This assignment was my first introduction to models. It was my first experience with the process of training and predicting a model, and I learned the importance of data by considering whether the model was overfitted when measuring accuracy.



#ChatGPT's Answer about Q2
k-Nearest Neighbors (kNN) and Random Forest (RF) are both widely used machine learning algorithms, but they differ fundamentally in their approach, performance characteristics, and optimal use cases. Here’s a detailed comparison:

### 1. **Algorithm Type and Approach**
   - **kNN**:
     - kNN is a **non-parametric**, **instance-based** (or lazy) learning algorithm. It doesn't explicitly learn a model; instead, it stores the training data and makes predictions by comparing a new instance to all training instances, selecting the \( k \) nearest neighbors based on a chosen distance metric (often Euclidean).
     - Prediction is typically based on the majority class among the neighbors (classification) or the average value (regression).
   - **Random Forest**:
     - Random Forest is a **parametric**, **ensemble** algorithm that builds multiple decision trees during training and aggregates their predictions. Each tree is trained on a subset of the data (with replacement, called bootstrapping), and a random subset of features is considered for splitting at each node.
     - Final predictions are based on the majority vote (classification) or average (regression) of all trees, helping to reduce variance and avoid overfitting.

### 2. **Training and Prediction Speed**
   - **kNN**:
     - **Training Time**: Very fast since kNN only stores the data without building a model.
     - **Prediction Time**: Relatively slow, especially on large datasets, since kNN computes the distance from a test point to every point in the training set.
   - **Random Forest**:
     - **Training Time**: Slower compared to kNN as it requires constructing many decision trees, but modern implementations (e.g., in `sklearn`) can be efficient.
     - **Prediction Time**: Faster than kNN for large datasets because predictions only require passing the instance through a fixed set of decision trees.

### 3. **Memory Requirements**
   - **kNN**: Requires storage of the entire dataset, which can lead to high memory usage, especially with large datasets.
   - **Random Forest**: Requires storage of the model (trees), but generally more memory-efficient as it doesn’t need to store all instances for prediction.

### 4. **Accuracy and Performance**
   - **kNN**: Performs well on smaller datasets with low-dimensional, well-separated classes. However, it struggles with high-dimensional or sparse data due to the **curse of dimensionality**.
   - **Random Forest**: Often achieves higher accuracy, particularly on complex or large datasets with high-dimensional features. Its ensemble approach helps it generalize better and reduce overfitting.

### 5. **Interpretability**
   - **kNN**: Simple to understand; the prediction is based directly on the nearest neighbors.
   - **Random Forest**: Harder to interpret due to the complex ensemble of trees, but **feature importance** scores can give insights into which features are most influential.

### 6. **Handling High-Dimensional Data**
   - **kNN**: Struggles with high-dimensional data, as it becomes challenging to find meaningful neighbors in many dimensions.
   - **Random Forest**: Handles high-dimensional data better because it randomly selects subsets of features for each tree, thus reducing dimensionality implicitly.

### 7. **Applications**
   - **kNN**: Often used in recommender systems, basic image and text classification, and cases where interpretability and simplicity are priorities.
   - **Random Forest**: Commonly used in structured/tabular data applications, credit scoring, medical diagnosis, and cases where predictive performance is prioritized over interpretability.

### 8. **Effectiveness on MNIST Dataset**
   - **kNN**: May perform adequately but can be inefficient on the full MNIST dataset due to high-dimensionality (784 features) and large size.
   - **Random Forest**: Tends to perform better on MNIST, achieving higher accuracy due to its robustness against high dimensionality and ability to capture complex patterns in the data.

### Summary Table

| Criterion                  | kNN                               | Random Forest                        |
|----------------------------|-----------------------------------|--------------------------------------|
| **Algorithm Type**         | Non-parametric, instance-based   | Parametric, ensemble (tree-based)    |
| **Training Time**          | Fast                             | Moderate                            |
| **Prediction Time**        | Slow                             | Fast                                |
| **Memory Requirements**    | High (stores dataset)            | Moderate (stores trees)             |
| **Curse of Dimensionality**| Affected                         | Less affected                       |
| **Accuracy**               | Decent for low-dimensional data  | Generally higher on complex data    |
| **Interpretability**       | High                             | Moderate                            |
| **Use Case**               | Small, low-dimensional datasets  | Large, high-dimensional datasets    |
| **Effectiveness on MNIST** | Moderate                         | High                                |

### Conclusion
For the MNIST dataset:
- **Random Forest** is generally more effective**, offering higher accuracy and faster prediction speeds after training, especially suitable given MNIST’s high-dimensional nature.
- **kNN** can work on MNIST but is typically less efficient due to its slower prediction time and sensitivity to high-dimensionality, which may reduce accuracy without dimension reduction.

In most practical cases, Random Forest outperforms kNN on MNIST in terms of both accuracy and scalability.